# Projeto de Engenharia do Conhecimento 2023/2024

*Projeto by: Renato Ferreira (58238), Pedro Lopes(58196), Simão Quintas (58190)*

### Index

1. Feature selection
    1. Using correlation
    2. Using stepwise methods
    3. Random Forests for Feature Selection
2. Principal Components analysis
    1. Linear PCA
    2. Kernel PCA
3. Model Tuning


## 1. Feature selection

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#from sklearn import tree
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

: 

Start the imputer and get the data

In [2]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5, weights="uniform")

data = pd.read_csv('proj-data.csv', na_values='?') #, na_values='?' crasha o bloco seguinte
print(data.columns)

Index(['age:', 'sex:', 'on thyroxine:', 'query on thyroxine:',
       'on antithyroid medication:', 'sick:', 'pregnant:', 'thyroid surgery:',
       'I131 treatment:', 'query hypothyroid:', 'query hyperthyroid:',
       'lithium:', 'goitre:', 'tumor:', 'hypopituitary:', 'psych:',
       'TSH measured:', 'TSH:', 'T3 measured:', 'T3:', 'TT4 measured:', 'TT4:',
       'T4U measured:', 'T4U:', 'FTI measured:', 'FTI:', 'TBG measured:',
       'TBG:', 'referral source:', 'diagnoses', '[record identification]'],
      dtype='object')


Let's drop the rows and columns with a large number of NA values and get dummies for the columns with strings

In [3]:
# Remover as linhas com pouca informação
data.dropna(thresh=3, subset=['T3 measured:', 'TT4 measured:', 'T4U measured:', 'FTI measured:', 'TBG measured:'], inplace=True)

# Remover as colunas com pouca informação
data.dropna(axis=1, thresh=3669, inplace=True)

# Remover as colunas que indicam se algo foi medido ou não e a que tem a indentificação
columns_to_drop = data.filter(like='measured').columns
data.drop(columns_to_drop, axis=1, inplace=True)
data.drop('[record identification]', axis=1, inplace=True)

# Remover linhas com homens grávidos
data = data[~((data['sex:'] == 'M') & (data['pregnant:'] == "t"))]

In [4]:
def transform_diagnoses(df):

    hyperthyroid_conditions = ['A', 'B', 'C', 'D']
    hypothyroid_conditions = ['E', 'F', 'G', 'H']
    binding_protein = ['I', 'J']
    general_health = ['K']
    replacement_therapy = ['L', 'M', 'N']
    discordant = ['R']
    other = ['O', 'P', 'Q', 'S', 'T']
    none = ['-']

    def replace_diagnoses(diagnoses_list):
        replaced_diagnoses = []
        for diag in diagnoses_list:
            if diag in hyperthyroid_conditions:
                replaced_diagnoses.append('diagnosed hyperthyroid')
            elif diag in hypothyroid_conditions:
                replaced_diagnoses.append('diagnosed hypothyroid')
            elif diag in binding_protein:
                replaced_diagnoses.append('diagnosed binding protein')
            elif diag in general_health:
                replaced_diagnoses.append('diagnosed general health')
            elif diag in replacement_therapy:
                replaced_diagnoses.append('diagnosed replacement therapy')
            elif diag in discordant:
                replaced_diagnoses.append('diagnosed antithyroid treatment')
            elif diag in other:
                replaced_diagnoses.append('diagnosed other')
            elif diag in none:
                replaced_diagnoses.append('diagnosed none')
            else:
                replaced_diagnoses.append('diagnosed other')

        return replaced_diagnoses

    df['diagnoses'] = df['diagnoses'].apply(lambda x: x.split("|") if "|" in x else [x])
    df['diagnoses'] = df['diagnoses'].apply(replace_diagnoses)
    
    return df


# Encode categorical variables
data = transform_diagnoses(data).explode('diagnoses')
print(data)
binary_cols = ['sex:', 'on thyroxine:', 'query on thyroxine:', 'on antithyroid medication:', 
               'sick:', 'pregnant:', 'thyroid surgery:', 'I131 treatment:', 'query hypothyroid:',
               'query hyperthyroid:', 'lithium:', 'goitre:', 'tumor:', 'hypopituitary:', 'psych:',
               'referral source:', 'diagnoses']

# Trocar os t e f e os diagnosticos para valores
data = pd.get_dummies(data, columns=binary_cols)

imputed_data = imputer.fit_transform(data)

df = pd.DataFrame(imputed_data, columns=data.columns)

      age: sex: on thyroxine: query on thyroxine: on antithyroid medication:  \
0       29    F             f                   f                          f   
1       29    F             f                   f                          f   
2       36    F             f                   f                          f   
3       60    F             f                   f                          f   
4       77    F             f                   f                          f   
...    ...  ...           ...                 ...                        ...   
7333    56    M             f                   f                          f   
7334    22    M             f                   f                          f   
7335    69    M             f                   f                          f   
7336    47    F             f                   f                          f   
7337    31    M             f                   f                          f   

     sick: pregnant: thyroid surgery: I

Let's make a simple evaluation function running 2 regression algorithms and producing the R2 for each

In [5]:
# train-test split
train, test = train_test_split(df, test_size=0.2, random_state=0)

def naive_model_testing(train, test):
    diagnoses_cols = [col for col in train.columns if 'diagnoses' in col]
    
    #test 2 models, DTs and LR, and print out the results
    dtr= DecisionTreeRegressor(max_depth=5)
    dtr.fit(train.drop(diagnoses_cols, axis=1), train[diagnoses_cols])

    lmr=LinearRegression()
    lmr.fit(train.drop(diagnoses_cols, axis=1), train[diagnoses_cols])

   # rf_preds=rfr.predict(X_test)
    dt_preds=dtr.predict(test.drop(diagnoses_cols, axis=1))
    lr_preds=lmr.predict(test.drop(diagnoses_cols, axis=1))

   # print("RVE RFs: %7.4f" % explained_variance_score(y_test, rf_preds))
    print("R2 Decision Tree Regression: %7.4f" % r2_score(test[diagnoses_cols], dt_preds))
    print("R2 Linear Regression: %7.4f" % r2_score(test[diagnoses_cols], lr_preds))

naive_model_testing(train, test)


R2 Decision Tree Regression:  0.4913
R2 Linear Regression:  0.1730


### Correlation 

As a first exercise we are going to use the Spearman correlation

In [ ]:
spear = df.corr(method='spearman')
spear

## 2. Principal Components Analysis

We are now going to use the [PCA module](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) from scikit-learn

First let's just find a 2D projection of our data (remember to use only the training set)


In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2).set_output(transform="pandas") #finding the two best PCs
pca.fit(train.drop('target', axis=1))
tve=0 #total variance explained
for i, ve in enumerate(pca.explained_variance_ratio_):
    tve+=ve
    print("PC%d - Variance explained: %7.4f - Total Variance: %7.4f" % (i, ve, tve) )
print()
print("Actual Eigenvalues:", pca.singular_values_)
for i,comp in enumerate(pca.components_):
    print("PC",i, "-->", comp)
    

### Exercise 3

1. Interpret the results above. 
   
2. What is the meaning of the PC vectors?

In [ ]:
# Exercise 3.1 
# A variância explicada de PC0 é maior, ou seja, este componente captura uma proporção significativamente maior da variabilidade total presente nos dados em comparação com os outros componentes.
# Actual Eigenvalues representam o tamanho dos principais vetores de componentes, mostrando que PC0 tem uma maior importância na variância.
# Em PC0 e PC1, as variáveis com maior valor absoluto contribuem mais para a definição do respetivo componente.

# Exercise 3.2 
# Os PC vectors representam as direções no espaço de características original ao longo das quais os dados variam mais. 
# Cada PC vector é uma combinação linear das variáveis originais, indicando quanto cada variável original contribui para essa direção ou eixo específico.

Now let's project the data using the principal components defined and use them for regression

In [ ]:
n_train=pca.transform(train.drop('target', axis=1))
n_test=pca.transform(test.drop('target', axis=1))
n_train['target'] = train['target']  # form the train dataframe to pass to the metrics function
n_test['target'] = test['target']  # form the test dataframe to pass to the metrics function
naive_model_testing(n_train, n_test)

quite poor results as expected

### A graphical view illustrated with binary classification data

We consider now the same data as a classification problem, assuming that patients with a target value of 250 or more means they have diabetes and with less that 250 they don't

In [ ]:
# with binary classified instances
# target values of 250 or more indicate diabetes
yc_diabetes=np.array([int(i>=250) for i in diabetes.target]) # to be used in graphics ahead

X_train, X_test, y_train, y_test = train_test_split(diabetes.data, yc_diabetes, test_size=0.2, random_state=23)
print("training set patients with target value >=250: ", (y_train).sum())
print("training set patients with target value <250: ", len(y_train) - (y_train).sum())

Let's plot the projection in 2 components 

In [ ]:
pca = PCA(n_components=2) #finding the two best PCs
pca.fit(X_train)
nX_train=pca.transform(X_train)
nX_test=pca.transform(X_test)
colors=np.array(["tab:blue", "tab:orange"])[y_train]
plt.scatter(nX_train[:,0], nX_train[:,1], c=colors)
plt.show()

also as a classification problem we can see it is hard to discriminate the two classes using only the two PCs

## 3. Model Tuning

For this example we are going to use Support Vector Classifiers, but any model learned so far can be used

We are going to use first [Scikit-Learn's GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html), an implementation of extensive parameter search. In its basic form it just requires:
* a bare bones model constructor 
* a dictionary containing the parameters to search for. The keys of the dictionary should correspond to the parameter to test and the values to a list of possible values to test
* a scoring function defining what is the criterion to select and rank the best models
* GridSearchCV uses by default 5-Fold Cross validation, but other validation criteria can be used

The result of GridSearchCV is a structure that contains the fitted models that can then be used for learning and application

Tet's try it with the C and gamma values for support vector classification

In [ ]:
from time import time
#from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import scipy.stats as stats

#make the dictionary with the testing parameters
#gammas = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7]
#Cs = [1, 10, 100, 1e3, 1e4, 1e5]
#param_grid = {'gamma': gammas, 'C': Cs}
depths = [3, 5, 10, 15]
m_sampl_split = [2, 5, 9]
prune_a = [0.0, 0.0001, 0.001, 0.01]
param_grid = {'max_depth': depths, 'min_samples_split': m_sampl_split, 'ccp_alpha': prune_a}

#define the model and do the grid search
#clf = SVC() # RBF (Gaussian) by default
clf = DecisionTreeClassifier(criterion='log_loss', random_state=23)
gs = GridSearchCV(estimator=clf, param_grid=param_grid, scoring="f1")

start = time()
gs=gs.fit(X_train, y_train)
print(
    'GridSearchCV took %.2f seconds for %d candidate parameter settings.'
    % ((time() - start), len(gs.cv_results_['params']))
)

Let's identify the best element parameters [best according to the scoring function, in this case it is the F1 score]

In [ ]:
#print('best gamma: %7.4f' % gs.best_estimator_.gamma)
#print('best C: %3.2f' %  gs.best_estimator_.C)
print('best maximum depth: %2.0f' % gs.best_estimator_.max_depth)
print('best minimum samples to split a node: %2.0f' %  gs.best_estimator_.min_samples_split)
print('best minimal cost pruning parameter: %1.4f' % gs.best_estimator_.ccp_alpha)

Just for sake of completion, we can use the best estimator model (the one with the optimized parameters) for prediction on the test set.

In [ ]:
preds=gs.best_estimator_.predict(X_test)
print('F1 : %7.4f' % f1_score(y_test, preds))
print('number of leaves:', gs.best_estimator_.get_n_leaves())

GridSearchCV gives you a number of statistics on the tests it runs:

In [ ]:
for i in gs.cv_results_.keys(): print(i)

We can print the results in a nice Pandas Data Frame

In [ ]:
grid_res = pd.DataFrame(gs.cv_results_)
grid_res.sort_values(by=['rank_test_score'], ascending=True, inplace=True) #sort the tested models by score
grid_res[['params', 'rank_test_score', 'mean_test_score', 'std_test_score', 'mean_fit_time', 'std_fit_time']] #show only mean and std of the test score

we can check if the 2nd best model produces different results 

In [ ]:
print('max_depth:', grid_res.loc[1, 'param_max_depth'],
      'min_samples_split:', grid_res.loc[1, 'param_min_samples_split'],
      'ccp_alpha:', '{:.2e}'.format(grid_res.loc[1, 'param_ccp_alpha']))
clf = DecisionTreeClassifier(criterion='log_loss', random_state=23,
                             max_depth=grid_res.loc[1, 'param_max_depth'],
                             min_samples_split=grid_res.loc[1, 'param_min_samples_split'],
                             ccp_alpha=grid_res.loc[1, 'param_ccp_alpha'])
clf.fit(X_train, y_train)
preds=clf.predict(X_test)
print('F1 : %7.4f' % f1_score(y_test, preds))
print('number of leaves:', clf.get_n_leaves())

Let's try now the RandomizedSearchCV and compare to the previous one.

In [ ]:
# configure randomized search (by default also 5-fold CV)
# notice the loguniform distributions

param_dist = {
#    'C': stats.loguniform(1, 1e5),
#    'gamma': stats.loguniform(1e-7, 1e-1),
    'max_depth': stats.randint(3, 16),
    'min_samples_split': stats.randint(2, 10),
    'ccp_alpha': stats.loguniform(1e-5, 0.01)
}

n_iter_search = 15
rs = RandomizedSearchCV(
    clf, param_distributions=param_dist, n_iter=n_iter_search
)

start = time()
rs = rs.fit(X_train, y_train)
print(
    'RandomizedSearchCV took %.2f seconds for %d candidates parameter settings'
    % ((time() - start), n_iter_search)
)

In [ ]:
print('best maximum depth: %2.0f' % rs.best_estimator_.max_depth)
print('best minimum samples to split a node: %2.0f' %  rs.best_estimator_.min_samples_split)
print('best minimal cost pruning parameter: %1.4f' % rs.best_estimator_.ccp_alpha)

Now we can use the best estimator model (the one with the optimized parameters) for prediction

In [ ]:
rs1 = rs.best_estimator_
rs1.fit(X_train, y_train)
preds=rs1.predict(X_test)
print('F1 : %7.4f' % f1_score(y_test, preds))
print('number of leaves:', rs1.get_n_leaves())

In [ ]:
rand_res = pd.DataFrame(rs.cv_results_)
rand_res.sort_values(by=['rank_test_score'], ascending= True, inplace=True) #sort the tested models by score
rand_res[['params', 'rank_test_score', 'mean_test_score', 'std_test_score', 'mean_fit_time', 'std_fit_time']] #show only mean and std of the test score

checking the 2nd best model 

In [ ]:
print('max_depth:', rand_res['param_max_depth'].iat[1],
      ', min_samples_split:', rand_res['param_min_samples_split'].iat[1],
      ', ccp_alpha:', '{:.2e}'.format(rand_res['param_ccp_alpha'].iat[1]))
clf = DecisionTreeClassifier(criterion='log_loss', random_state=23,
                             max_depth=rand_res['param_max_depth'].iat[1],
                             min_samples_split=rand_res['param_min_samples_split'].iat[1],
                             ccp_alpha=rand_res['param_ccp_alpha'].iat[1])
clf.fit(X_train, y_train)
preds=clf.predict(X_test)
print('F1 : %7.4f' % f1_score(y_test, preds))
print('number of leaves:', clf.get_n_leaves())


### Exercise 4
1. Discuss the values above in terms of coherency of the parameters found. Do you find a pattern in the best values for max_dept and ccp_alpha?
2. Compare the first 3 models results using the testing set and discuss your findings [Optional]


In [ ]:
# Exercise 4.1
#Enquanto que o valor para o max_dept parece ser consistentemente 3, o valor para ccp_aplpha vaira bastante


In [ ]:
# Exercise 4.2



In [ ]:
# Comments on results of Exercise 4.2

